# Engineering Database
### Last updated: 06/23/2021

**Purpose:** Wrangle data to desired schema outlined on ALS - Data Engineering Exercise producing a people.csv and acquisition_facts.csv file.

In [84]:
import boto3
import pandas as pd
import os

In [85]:
# enter your working directory path
wd = ''
os.chdir = wd

In [8]:
# retrieve your datasets from AWS
client = boto3.client('s3')

info_path = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv'
email_path = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv'
subs_path = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv'

info_df = pd.read_csv(info_path)
email_df = pd.read_csv(email_path)
subs_df = pd.read_csv(subs_path)



In [10]:
# display your datasets
display(info_df.head(n = 2))
print(info_df.columns)

display(email_df.head(n = 2))
print(email_df.columns)

display(subs_df.head(n = 2))
print(subs_df.columns)

,cons_id,prefix,firstname,middlename,lastname,suffix,salutation,gender,birth_dt,title,...,change_password_next_login,consent_type_id,create_dt,create_app,create_user,modified_dt,modified_app,modified_user,status,note
0,1,NaN,NaN,Lee,NaN,MD,NaN,E,NaN,vSkSIzEQJdXnqeTTTXSG,...,0,5958,"Fri, 1983-08-26 06:02:03",1484,6162,"Sun, 2015-12-27 09:28:02",4022,6349,1,NaN
1,2,NaN,NaN,NaN,NaN,II,boFqBKgLlSgEZsFrgCZd,E,"Mon, 2004-11-15",NaN,...,1,4236,"Mon, 1979-03-05 21:08:54",4176,5476,"Tue, 1989-06-20 13:28:57",9010,5698,1,NaN


Index(['cons_id', 'prefix', 'firstname', 'middlename', 'lastname', 'suffix',
       'salutation', 'gender', 'birth_dt', 'title', 'employer', 'occupation',
       'income', 'source', 'subsource', 'userid', 'password', 'is_validated',
       'is_banned', 'change_password_next_login', 'consent_type_id',
       'create_dt', 'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note'],
      dtype='object')


,cons_email_id,cons_id,cons_email_type_id,is_primary,email,canonical_local_part,domain,double_validation,create_dt,create_app,create_user,modified_dt,modified_app,modified_user,status,note
0,1,548198,3361,1,xmartinez@vincent.com,NaN,gmail.com,NaN,"Wed, 1994-01-26 23:49:16",4072,9954,"Sat, 2014-04-19 19:10:39",1990,7595,1,NaN
1,2,491137,2474,1,hmiller@haynes.biz,jqCyozTDojYuylQPTHfm,hotmail.com,NaN,"Thu, 1999-12-09 06:18:27",1600,5716,"Sat, 1984-07-14 05:55:27",4686,3248,1,NaN


Index(['cons_email_id', 'cons_id', 'cons_email_type_id', 'is_primary', 'email',
       'canonical_local_part', 'domain', 'double_validation', 'create_dt',
       'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note'],
      dtype='object')


,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub,unsub_dt,modified_dt
0,1,332188,1,1,"Sat, 1971-06-12 15:38:44","Thu, 1990-06-28 10:54:20"
1,2,536526,1,1,"Wed, 2006-07-12 01:50:45","Thu, 1979-09-20 06:02:35"


Index(['cons_email_chapter_subscription_id', 'cons_email_id', 'chapter_id',
       'isunsub', 'unsub_dt', 'modified_dt'],
      dtype='object')


# Exercise 1

In [48]:
# create a table that only includes primary addresses
temp_email = email_df.loc[email_df['is_primary'] == 1]


In [49]:
# filter to only include subscription statuses where chapter_id = 1
temp_subs = subs_df.loc[subs_df['chapter_id'] == 1]


In [50]:
temp_subs.shape

(275484, 6)

Mapping out the schema above, and looking at the shared id columns across each tables, we identify the columns we are interested in: 



| new_column_name      | old_column_name | table | notes |
| ----------- | ----------- | ----------- | ----------- |
|cons_id    |        |  email       |     ID to info    |
|email    |email         |email         |         |
|created_dt    |create_dt         |email         |         |
|cons_email_id    |         |email         |ID to subs         |
|      |     |      |     |
|cons_id    |         |info         |ID to subs         |
|code    |source         |info         |         |
|      |     |      |     |
|cons_email_id    |         |         |ID to info         |
|is_unsub    |isunsub         |subs         |         |
|updated_dt    |modified_dt         |subs         |         |


In [51]:
# filter your datasets to only include columns you are interested in using
temp_email = temp_email[['cons_email_id', 'cons_id', 'email', 'create_dt']]

temp_info = info_df[['cons_id', 'source']]

temp_subs = temp_subs[['cons_email_id', 'isunsub', 'modified_dt']]

In [52]:
# merge datasets together keeping only the ids that are found in both datasets 
# we can assume that we are only interested in the subscriptions identified as primary email addresses 

df = pd.merge(temp_email, temp_info, on = 'cons_id',  how = 'inner')

In [53]:
df = pd.merge(df, temp_subs, on = 'cons_email_id', how = 'inner')

In [54]:
# If an email is not present in the subscription table, it is assumed that they are still subscribed if chapter ID = 1
df['modified_dt'].isnull().sum()


0

In [55]:
df.head()

,cons_email_id,cons_id,email,create_dt,source,isunsub,modified_dt
0,3,413429,aaron64@yahoo.com,"Wed, 1992-11-18 16:46:27",NaN,1,"Sun, 1977-10-02 12:32:10"
1,4,347346,wyattvincent@hotmail.com,"Sat, 1983-11-26 16:49:14",NaN,1,"Wed, 2003-05-28 02:47:44"
2,5,443000,tspencer@hotmail.com,"Wed, 2000-11-15 13:28:34",twitter,1,"Tue, 2000-04-11 15:20:13"
3,6,412213,ogarcia@gmail.com,"Thu, 1984-03-08 11:40:21",NaN,1,"Sat, 1992-07-18 17:35:23"
4,9,603436,madeline69@mccarthy-jackson.com,"Mon, 1976-06-28 08:58:19",twitter,1,"Thu, 1999-02-04 05:46:24"


In [57]:
# modify the column names
col_dict = {'source': 'code', 'isunsub': 'is_unsub', 'create_dt': 'created_dt', 'modified_dt': 'updated_dt'}
df = df.rename(columns = col_dict )

In [68]:
df.head()

,cons_email_id,cons_id,email,created_dt,code,is_unsub,updated_dt
0,3,413429,aaron64@yahoo.com,1992-11-18 16:46:27,NaN,1,1977-10-02 12:32:10
1,4,347346,wyattvincent@hotmail.com,1983-11-26 16:49:14,NaN,1,2003-05-28 02:47:44
2,5,443000,tspencer@hotmail.com,2000-11-15 13:28:34,twitter,1,2000-04-11 15:20:13
3,6,412213,ogarcia@gmail.com,1984-03-08 11:40:21,NaN,1,1992-07-18 17:35:23
4,9,603436,madeline69@mccarthy-jackson.com,1976-06-28 08:58:19,twitter,1,1999-02-04 05:46:24


In [67]:
# clean up your dates to a standardized format

# remove the day of the week at the start. of each date
df['updated_dt'] = df['updated_dt'].str.split(', ').str[-1].str.strip()
df['created_dt'] = df['created_dt'].str.split(', ').str[-1].str.strip()



In [71]:
# convert data to proper datetime object
df['created_dt'] = pd.to_datetime(df['created_dt'], format = '%Y-%m-%d %H:%S:%f')
df['created_dt'] = pd.to_datetime(df['created_dt'], format = '%Y-%m-%d %H:%S:%f')


In [79]:
# drop id columns you don't need anymore
df = df.drop(columns = ['cons_email_id', 'cons_id'])

In [80]:
df.head()

,email,created_dt,code,is_unsub,updated_dt
0,aaron64@yahoo.com,1992-11-18 16:00:46.270,NaN,1,1977-10-02 12:00:32.100
1,wyattvincent@hotmail.com,1983-11-26 16:00:49.140,NaN,1,2003-05-28 02:00:47.440
2,tspencer@hotmail.com,2000-11-15 13:00:28.340,twitter,1,2000-04-11 15:00:20.130
3,ogarcia@gmail.com,1984-03-08 11:00:40.210,NaN,1,1992-07-18 17:00:35.230
4,madeline69@mccarthy-jackson.com,1976-06-28 08:00:58.190,twitter,1,1999-02-04 05:00:46.240


In [82]:
df.to_csv('people.csv', index = False)

# Exercise 2

In [86]:
# check if there are duplicate emails (just in case)
df['email'].duplicated().any()

False

In [100]:
# convert created datetime to only include the calendar date (%Y-%m-%d)
df = df.assign(created_dt = df['created_dt'].dt.round('D'))


In [106]:
# group by created_dt and count number of unique constitutents
acquisition = df.groupby('created_dt')['email'].nunique().reset_index()

In [108]:
# rename columns to match desired schema
col_rename_dict = {'created_dt': 'acquisition_date', 'email':'acquisitions'}
acquisition = acquisition.rename(columns = col_rename_dict)

In [109]:
acquisition.head()

,acquisition_date,acquisitions
0,1970-01-01,6
1,1970-01-02,12
2,1970-01-03,5
3,1970-01-04,8
4,1970-01-05,3


In [110]:
# save as a csv file
acquisition.to_csv('acquisition_facts.csv', index = False)